# Step 2.2: Getting Speech to Noise Ratio

This code will get you the Speech-to-Noise Ratio for audio files. This process was really difficult to make work, so be sure to follow directions carefully. I did not develop this code myself. It is adapted directly from sources which I will cite below. Huge thank you to Rémi Rigal ([LinkedIn](https://www.linkedin.com/in/r%C3%A9mi-rigal-02b798142/?originalSubdomain=fr), [GitHub](https://github.com/RemiRigal)) and John Meade ([LinkedIn](https://www.linkedin.com/in/jgmeade/), [GitHub](https://gist.github.com/johnmeade)) for taking the time for personal correspondence and troubleshooting their code with me!

**Note:** I will provide two options here. You can perform either or both. By default, this will produce a dataframe with the results from both options for checking. You must install *snreval* with pip or other methods for Option \#2 to work. Option 2 will also give you the NIST STNR (https://labrosa.ee.columbia.edu/~dpwe/tmp/nist/doc/stnr.txt).

## Required Packages

The following packages are necessary to run this code:
os, [pandas](https://pypi.org/project/pandas/), [numpy](https://pypi.org/project/numpy/), [soundfile](https://pypi.org/project/SoundFile/), [snreval](https://pypi.org/project/snreval/), [interruptingcow](https://pypi.org/project/interruptingcow/)

## References:

**Option 1:** The *wada_snr* function is the work of [John Meade](https://gist.github.com/johnmeade) and is taken from https://gist.github.com/johnmeade/d8d2c67b87cda95cd253f55c21387e75.

**Option 2:** The *snreval* code is the work of [Rémi Rigal](https://github.com/RemiRigal) and is taken from https://github.com/RemiRigal/snreval-python.

## Initial Set-Up for Both Options

In [ ]:
import pandas as pd

#this is the filepath where the gold standard .csv file is stored
aint_gold_standard_csv_path = "path"

be_gold_standard_csv_path = "path"

done_gold_standard_csv_path = "path"


#This will create the dataframe from the csv
aint_gs_df = pd.read_csv(aint_gold_standard_csv_path)

be_gs_df = pd.read_csv(be_gold_standard_csv_path)

done_gs_df = pd.read_csv(done_gold_standard_csv_path)



#this is the path where the .wav files are stored
aint_wav_path = "path"

be_wav_path = "path"

done_wav_path = "path"

# Option 1: snr.py

The *wada_snr* function is the work of [John Meade](https://gist.github.com/johnmeade) and is taken from https://gist.github.com/johnmeade/d8d2c67b87cda95cd253f55c21387e75.

In [ ]:
# import required packages
import os
import pandas as pd
import numpy as np
import soundfile as sf

## Define the SNR Getting Function

This function takes the following arguments:

<ol>
<li>The filepath to a wav file</li>
</ol>

In [ ]:
def wada_snr(wav):
    # Direct blind estimation of the SNR of a speech signal.
    #
    # Paper on WADA SNR:
    #   http://www.cs.cmu.edu/~robust/Papers/KimSternIS08.pdf
    #
    # This function was adapted from this matlab code:
    #   https://labrosa.ee.columbia.edu/projects/snreval/#9
    #
    # MIT license, John Meade, 2020

    """
    Reads in a numpy arrary and produces the WADA SNR
    """

    #Notes from John
    """
    the WADA algorithm appears to perform consistently across different samplerates, but I have only briefly tested that.

    Also, the algo appears to have pretty common edge-cases. If you get exactly -20dB or 100dB SNR you may want to discard those results, they might be kind of undefined
    """

    # init
    eps = 1e-10
    
    # next 2 lines define a fancy curve derived from a gamma distribution -- see paper
    db_vals = np.arange(-20, 101)
    
    g_vals = np.array([0.40974774, 0.40986926, 0.40998566, 0.40969089, 0.40986186, 0.40999006, 0.41027138, 0.41052627, 0.41101024, 0.41143264, 0.41231718, 0.41337272, 0.41526426, 0.4178192 , 0.42077252, 0.42452799, 0.42918886, 0.43510373, 0.44234195, 0.45161485, 0.46221153, 0.47491647, 0.48883809, 0.50509236, 0.52353709, 0.54372088, 0.56532427, 0.58847532, 0.61346212, 0.63954496, 0.66750818, 0.69583724, 0.72454762, 0.75414799, 0.78323148, 0.81240985, 0.84219775, 0.87166406, 0.90030504, 0.92880418, 0.95655449, 0.9835349 , 1.01047155, 1.0362095 , 1.06136425, 1.08579312, 1.1094819 , 1.13277995, 1.15472826, 1.17627308, 1.19703503, 1.21671694, 1.23535898, 1.25364313, 1.27103891, 1.28718029, 1.30302865, 1.31839527, 1.33294817, 1.34700935, 1.3605727 , 1.37345513, 1.38577122, 1.39733504, 1.40856397, 1.41959619, 1.42983624, 1.43958467, 1.44902176, 1.45804831, 1.46669568, 1.47486938, 1.48269965, 1.49034339, 1.49748214, 1.50435106, 1.51076426, 1.51698915, 1.5229097 , 1.528578  , 1.53389835, 1.5391211 , 1.5439065 , 1.54858517, 1.55310776, 1.55744391, 1.56164927, 1.56566348, 1.56938671, 1.57307767, 1.57654764, 1.57980083, 1.58304129, 1.58602496, 1.58880681, 1.59162477, 1.5941969 , 1.59693155, 1.599446  , 1.60185011, 1.60408668, 1.60627134, 1.60826199, 1.61004547, 1.61192472, 1.61369656, 1.61534074, 1.61688905, 1.61838916, 1.61985374, 1.62135878, 1.62268119, 1.62390423, 1.62513143, 1.62632463, 1.6274027 , 1.62842767, 1.62945532, 1.6303307 , 1.63128026, 1.63204102])

    
    # peak normalize, get magnitude, clip lower bound
    wav = np.array(wav)
    
    wav = wav / abs(wav).max()
    
    abs_wav = abs(wav)
    
    abs_wav[abs_wav < eps] = eps

    # calcuate statistics
    # E[|z|]
    v1 = max(eps, abs_wav.mean())
    
    # E[log|z|]
    v2 = np.log(abs_wav).mean()
    
    # log(E[|z|]) - E[log(|z|)]
    v3 = np.log(v1) - v2

    
    # table interpolation
    wav_snr_idx = None
    
    if any(g_vals < v3):
        
        wav_snr_idx = np.where(g_vals < v3)[0].max()
        
    # handle edge cases or interpolate
    if wav_snr_idx is None:
        
        wav_snr = db_vals[0]
        
    elif wav_snr_idx == len(db_vals) - 1:
        
        wav_snr = db_vals[-1]
        
    else:
        
        wav_snr = db_vals[wav_snr_idx] + \
            (v3-g_vals[wav_snr_idx]) / (g_vals[wav_snr_idx+1] - \
            g_vals[wav_snr_idx]) * (db_vals[wav_snr_idx+1] - db_vals[wav_snr_idx])

        
    # Calculate SNR
    dEng = sum(wav**2)
    
    dFactor = 10**(wav_snr / 10)
    
    dNoiseEng = dEng / (1 + dFactor) # Noise energy
    
    dSigEng = dEng * dFactor / (1 + dFactor) # Signal energy
    
    snr = 10 * np.log10(dSigEng / dNoiseEng)

    return snr

## Adding the WADA Speech-to-Noise Ratio Column to the Dataframe

### Feature: Ain't

In [ ]:
#creates a new column for WADA_SNR filled with 
# nan values which will be replaced
aint_gs_df["WadaSNRMeade"] = np.nan

#loops through the rows of the gold standard dataframe
for file_row in aint_gs_df.itertuples():

    #this will convert a .wav audio file to a numpy arrary
    wav, samplerate = sf.read(
        f"{aint_wav_path}/16khz_{file_row.File}_Line{file_row.Line}_FeatCount{file_row.FeatureCountPerLine}.wav")
    
    #this will return the WADA Speech-to-noise Ratio
    # and replace the row's nan value in the WadaSNRMeade
    # column with the WADA SNR value
    aint_gs_df.loc[file_row.Index, "WadaSNRMeade"] = float(wada_snr(wav))

### Feature: Be

In [ ]:
#creates a new column for WADA_SNR filled with 
# nan values which will be replaced
be_gs_df["WadaSNRMeade"] = np.nan

#loops through the rows of the gold standard dataframe
for file_row in be_gs_df.itertuples():

    #this will convert a .wav audio file to a numpy arrary
    wav, samplerate = sf.read(
        f"{be_wav_path}/16khz_{file_row.File}_Line{file_row.Line}_FeatCount{file_row.FeatureCountPerLine}.wav")
    
    #this will return the WADA Speech-to-noise Ratio
    # and replace the row's nan value in the WadaSNRMeade
    # column with the WADA SNR value
    be_gs_df.loc[file_row.Index, "WadaSNRMeade"] = float(wada_snr(wav))

### Feature: Done

In [ ]:
#creates a new column for WADA_SNR filled with 
# nan values which will be replaced
done_gs_df["WadaSNRMeade"] = np.nan

#loops through the rows of the gold standard dataframe
for file_row in done_gs_df.itertuples():

    #this will convert a .wav audio file to a numpy arrary
    wav, samplerate = sf.read(
        f"{done_wav_path}/16khz_{file_row.File}_Line{file_row.Line}_FeatCount{file_row.FeatureCountPerLine}.wav")
    
    #this will return the WADA Speech-to-noise Ratio
    # and replace the row's nan value in the WadaSNRMeade
    # column with the WADA SNR value
    done_gs_df.loc[file_row.Index, "WadaSNRMeade"] = float(wada_snr(wav))

# Option 2: SNReval

The *snreval* code is the work of [Rémi Rigal](https://github.com/RemiRigal) and is taken from https://github.com/RemiRigal/snreval-python.

## Installation Directions

This option is more complicated to install. Here are the things I had to do to make it work:

<ol>
    <li>At the time of this installation, I was currently using macOS BigSur 11.2.3 and python3.9.2</li>
    <li> Go here <a href="https://labrosa.ee.columbia.edu/projects/snreval/#9">https://labrosa.ee.columbia.edu/projects/snreval/#9</a> and install the MATLAB Compiler Runtime ("MACI64 MCR Installer") and download and unzip the "snreval_MACI64.zip" </li>
    <li>When I installed the MATLAB Compiler Runtime, it installed as  a folder rather than an app, so I had to change a line in the code to rule compiler:
        <ul>
            <li>In the "snreval_MACI64" folder that you downloaded, you'll see a file called "run_snreval_prj.sh".</li> 
            <li>Open that in a text editor, and change line 14 from "MCRROOT=/Applications/MATLAB_R2010b.app" to "MCRROOT=/Applications/MATLAB/MATLAB_Compiler_Runtime/v714". This should direct the run file to the right folder</li>
        </ul>
    </li>
    <li>One other thing that was missing was something called XQuartz. Apparently, Mac used to ship with X11 code (not sure what it is), but now it doesn't. The SNReval needs this to run, so install it from here: <a href="https://www.xquartz.org/">https://www.xquartz.org/</a>. Once you have that, everything should be ready!</li>
</ol>

In [ ]:
# import required packages
import os
import pandas as pd
import numpy as np
import soundfile as sf
from interruptingcow import timeout

#the following assumes you have installed snreval as laid out above
from snreval import SNREval

## Define the SNR Getting Function

This function takes the following arguments:

<ol>
<li>The gold standard dataframe (created in the initial set-up above</li>
<li>The filepath to a wav file</li>
</ol>

In [ ]:
def get_Rigal_df(gs_df, wav_path):

    #loops through the rows of the gold standard dataframe
    for file_row in gs_df.itertuples():

        #the code was getting stuck and running endlessly on single files
        #  I calculated the average time for this code to process the CORAAL parsed files, 
        #  which was about 5 seconds. So, this will try to process the file,
        #  and if it takes more than 10 seconds, it will skip it and write a 
        #  NaN to the SNR columns
        try:

            #sets the timeout timer to 10 seconds
            with timeout(10, exception = RuntimeError):

                #this will create an instance of the class SNREval
                # essentially, this will get the process up and running
                snr_eval = SNREval(f"{snreval_path}")

                #this will create a dataframe that will give you the 
                # WADA SNR and NIST STNR
                snr_df = snr_eval.eval(f"{wav_path}/16khz_{file_row.File}_Line{file_row.Line}_FeatCount{file_row.FeatureCountPerLine}.wav")

                #this will return the WADA Speech-to-noise Ratio
                # and replace the row's nan value in the WadaSNR
                # column with the WADA SNR value
                gs_df.loc[file_row.Index, "WadaSNRRigal"] = float(snr_df.iloc[0]['SNR'])

                #this will return the NIST STNR
                # and replace the row's nan value in the NistSNRRigal
                # column with the NistSNRRigal value
                gs_df.loc[file_row.Index, "NistSNRRigal"] = float(snr_df.iloc[0]['STNR'])
                
                # enable this if you want to see the progress in real time
                print(f"{file_row.File}--{file_row.Line}")

        except RuntimeError: 

            #writes a NaN to the row
            gs_df.loc[file_row.Index, "WadaSNRRigal"] = ""

            #writes a NaN to the row
            gs_df.loc[file_row.Index, "NistSNRRigal"] = ""     

            #prints a flag in case you want to see exactly where things are
            #getting stuck
            print(f"{file_row.File}--{file_row.Line} got stuck and has been skipped.")

            #passes to the next iteration
            pass 
        
    return gs_df

## Adding the WADA and NIST Speech-to-Noise Ratio Columns to the Dataframe

In [ ]:
#this is the path to the unzipped snreval_MACI64 folder
snreval_path = "path"

### Feature: Ain't

In [ ]:
#creates new columns for WADA_SNR and NIST_STNR filled with 
# nan values which will be replaced
aint_gs_df["WadaSNRRigal"] = np.nan
aint_gs_df["NistSNRRigal"] = np.nan

# executes the function
aint_gs_df = get_Rigal_df(aint_gs_df, aint_wav_path)

### Feature: Be

In [ ]:
#creates new columns for WADA_SNR and NIST_STNR filled with 
# nan values which will be replaced
be_gs_df["WadaSNRRigal"] = np.nan
be_gs_df["NistSNRRigal"] = np.nan

# executes the function
be_gs_df = get_Rigal_df(be_gs_df, be_wav_path)

### Feature: Done

In [ ]:
#creates new columns for WADA_SNR and NIST_STNR filled with 
# nan values which will be replaced
done_gs_df["WadaSNRRigal"] = np.nan
done_gs_df["NistSNRRigal"] = np.nan

# executes the function
done_gs_df = get_Rigal_df(done_gs_df, done_wav_path)

## Sorting the Dataframes by File and Line

This will sort the dataframes first by filename and then by line number. Doing this each step will ensure consistency across the board.

### Feature: Ain't

In [ ]:
aint_gs_df = aint_gs_df.sort_values(by=['File', 'Line'])

### Feature: Be

In [ ]:
be_gs_df = be_gs_df.sort_values(by=['File', 'Line'])

### Feature: Done

In [ ]:
done_gs_df = done_gs_df.sort_values(by=['File', 'Line'])

## Exporting Dataframes to CSV Files

This will export the dataframes to CSV files.

In [ ]:
# Designate the output path where the CSVs will be stored
csv_output_path = "path"

### Feature: Ain't

In [ ]:
aint_gs_df.to_csv(f"{csv_output_path}aint_variations_SNR.csv", index=False)

### Feature: Be

In [ ]:
be_gs_df.to_csv(f"{csv_output_path}be_SNR.csv", index=False)

### Feature: Done

In [ ]:
done_gs_df.to_csv(f"{csv_output_path}done_SNR.csv", index=False)